In [ ]:
import pandas as pd
import nltk
import numpy as np
nltk.download('stopwords')
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from nltk.stem import SnowballStemmer
import time
import pickle

#Visualization
import matplotlib.pyplot as plt

#Feature Engineering
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
#Evaluation Metric
from sklearn.metrics import accuracy_score, confusion_matrix,f1_score, precision_score,recall_score,classification_report
import seaborn as sns
from scipy.sparse import csr_matrix

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
a = [i for i in range(1000, 10000, 500)]
print(a)

[1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6000, 6500, 7000, 7500, 8000, 8500, 9000, 9500]


In [ ]:
def calc_time_svm():
  results = []
  for i in a:
    df = pd.read_csv(f"https://media.githubusercontent.com/media/PTIT-Assignment-Projects/ai-svm-email-spam/refs/heads/main/dataset/sampled_dataset{i}.csv")
    def remove_special_characters(word):
        return re.sub(r'[^a-zA-Z\s]', '', word)
    ENGLISH_STOP_WORDS = set(stopwords.words('english'))
    def remove_stop_words(words):
        return [word for word in words if word not in ENGLISH_STOP_WORDS]
    def remove_url(word):
        return re.sub(r"http\S+", "", word)
    df['text'] = df['text'].apply(remove_special_characters)
    df['text'] = df['text'].apply(remove_url)
    df['text'] = df['text'].apply(word_tokenize)
    df['text'] = df['text'].apply(remove_stop_words)
    df['text'] = df['text'].apply(' '.join)
    stemmer = SnowballStemmer('english')
    def stem_text(text):
        tokens = nltk.word_tokenize(text)
        stemmed_tokens = [stemmer.stem(token) for token in tokens]
        return ' '.join(stemmed_tokens)
    df['text'] = df['text'].apply(stem_text)
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(df['text'])
    y = df['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    result = []
    svm_base = SVC(kernel = 'linear')
    start_time = time.time()
    svm_base.fit(X_train, y_train)
    end_time = time.time()
    y_pred = svm_base.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    result.append({
            'class_name': svm_base.__class__.__name__,
            'time': end_time - start_time,
            'accuracy_score': accuracy,
            'f1_score': f1
        })
    results_df = pd.DataFrame(result)
    results_df.to_csv(f'base_svm_tfidf_{i}.csv', index=False)
  return results

In [ ]:
calc_time_svm()

KeyboardInterrupt: 